# Settings

In [ ]:
pip install transformers datasets timm --upgrade deepl openpyxl torchmetrics

  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Bachelorarbeit/Dataset

dataset_path = "/content/drive/MyDrive/Bachelorarbeit/Dataset/slogans_extended.csv"

Mounted at /content/drive
/content/drive/MyDrive/Bachelorarbeit/Dataset


In [ ]:
from transformers import DetrFeatureExtractor, DetrForObjectDetection, pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
import torch
from PIL import Image
import glob
import random
from pprint import pprint
from tqdm import trange
import pandas as pd
import numpy as np
from torchmetrics.text.rouge import ROUGEScore
from pprint import pprint


# Object Detection

In [ ]:
feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-101")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-101")

Downloading:   0%|          | 0.00/274 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243M [00:00<?, ?B/s]

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet101_a1h-36d3f2aa.pth" to /root/.cache/torch/hub/checkpoints/resnet101_a1h-36d3f2aa.pth


In [ ]:
DETR_OUTPUT = set()

for img in glob.glob("/content/drive/MyDrive/Bachelorarbeit/Dataset_Vision/Images/*.*"): 
    image = Image.open(img)

    inputs = feature_extractor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    target_sizes = torch.tensor([image.size[::-1]])
    results = feature_extractor.post_process(outputs, target_sizes=target_sizes)[0]


    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = torch.tensor([[round(i, 2) for i in box.tolist()]])
        
        if score >0.7:
          DETR_OUTPUT.add(str(model.config.id2label[label.item()]))



/usr/local/lib/python3.7/dist-packages/transformers/models/detr/feature_extraction_detr.py:854: FutureWarning: `post_process` is deprecated and will be removed in v5 of Transformers, please use `post_process_object_detection`
  FutureWarning,


In [ ]:
print(DETR_OUTPUT)

{'bottle', 'sandwich', 'person', 'dining table'}


# Zero-Shot Sense Classifier

In [ ]:

classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
context = set()

for item in DETR_OUTPUT:
  sense_labels = ['taste', 'touch', 'hearing', 'sight', 'smell']
  out_prob = classifier(item, sense_labels, multi_label=True)
  for label in out_prob:
    context.add(out_prob["labels"][0])
    context.add(out_prob["labels"][1])
print(context)

{'taste', 'touch', 'hearing'}


In [ ]:
context.update(DETR_OUTPUT)
print(context)

{'bottle', 'sandwich', 'person', 'dining table', 'taste', 'touch', 'hearing'}


In [ ]:
context_dict = {}

for i in range(3):
  l = list(context)
  sampled_l = random.sample(l, k=len(l))
  context_string = " ".join(sampled_l)
  context_dict.update({f"conetxt_{i}": context_string})

pprint(context_dict)

{'conetxt_0': 'taste sandwich touch person hearing bottle dining table',
 'conetxt_1': 'touch hearing sandwich person taste bottle dining table',
 'conetxt_2': 'hearing person sandwich taste dining table touch bottle'}


# Slogan Generation

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Bachelorarbeit/GPTNEO-Model").cuda()
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Bachelorarbeit/GPTNEO-Model")

In [ ]:
def sample_sequence(model, length, context_in, segments_tokens=None, num_samples=1, temperature=1, repetition_penalty=1.0, device='cpu'):
    context_in = torch.tensor(context_in, dtype=torch.long, device=device)
    context_in = context_in.unsqueeze(0).repeat(num_samples, 1)
    generated = context_in

    with torch.no_grad():
        for _ in trange(length):

            inputs = {'input_ids': generated}
            if segments_tokens != None:
              inputs['token_type_ids'] = torch.tensor(segments_tokens[:generated.shape[1]]).unsqueeze(0).repeat(num_samples, 1)


            outputs = model(**inputs) 
            next_token_logits = outputs[0][:, -1, :] / (temperature if temperature > 0 else 1.)

            # repetition penalty from CTRL (https://arxiv.org/abs/1909.05858)
            for i in range(num_samples):
                for _ in set(generated[i].tolist()):
                    next_token_logits[i, _] /= repetition_penalty
                
    return generated

In [ ]:
input_id_list = []
for context_words in context_dict.values():

  context_tkn = tokenizer.additional_special_tokens_ids[0]
  slogan_tkn = tokenizer.additional_special_tokens_ids[1]

  input_ids = [context_tkn] + tokenizer.encode(context_words)

  segments = [slogan_tkn] * 64
  segments[:len(input_ids)] = [context_tkn] * len(input_ids)
  input_ids += [slogan_tkn]
  input_id_list.append(input_ids)


In [ ]:
model.to(torch.device('cpu'))

reference = []

for i in range(len(input_id_list)):
  generated = sample_sequence(model, length=20, context_in=input_id_list[i], segments_tokens=segments)
  sample_outputs = model.generate(generated, do_sample=True, max_length=20, top_k=10 ,top_p=0.80, temperature=1., num_return_sequences=20,repetition_penalty=2.0)

  for i, sample_output in enumerate(sample_outputs):
    decoded = tokenizer.decode(sample_output, skip_special_tokens=False)
    decoded = decoded.split("<|endoftext|>")[0].split("<slogan>")[1]
    reference.append(decoded)
    print(f"{i}: {decoded}")



# DeepL implementation

In [ ]:
dataset = pd.DataFrame({"slogans": reference})

if dataset.duplicated().any():
  print("Dataset contains duplicates!")
else:
  print("No duplicates in dataset")

print("Count Rows: ",dataset.count())
dataset = dataset.drop_duplicates()
print("Count Rows after drop duplicates: ",dataset.count())

In [ ]:
import deepl

auth_key = "a3ad6928-7b1b-5b3e-60cd-70a5e4cda6db:fx"
translator = deepl.Translator(auth_key)

ger_slogans = []

for slogan in dataset.iterrows():
  result = translator.translate_text(str(slogan[1][0]), target_lang="DE")
  ger_slogans.append(result.text)
  print(f"{result.text}")


# Saved Slogans

In [ ]:

slogan_dict = {"ENG_Slogans": reference,
               "GER_Slogans": ger_slogans}

df = pd.DataFrame(slogan_dict)

with pd.ExcelWriter('/content/drive/MyDrive/Bachelorarbeit/Generated Slogans/sense_slogans.xlsx', engine="openpyxl")as writer:  
  df.to_excel(writer)

#Evaluate

In [ ]:
ds = pd.read_csv('./slogans_extended.csv')["slogan"]
ds = ds.drop_duplicates()
for i in range(len(reference)):
  new_ds = ds.loc[ds.str.contains(reference[i], case=False,na=False)]
  if new_ds.any():  
    print(f"Slogan: {reference[i]} is represented in the Dataset at row {new_ds.index[0]}")
  else:
    print(f"Slogan: {reference[i]} is not in Dataset")

## ROUGE Score

In [ ]:
candidates = random.sample([str(slogan) for slogan in reference], k=60)
shuffeld_reference = random.sample(list(ds), k=60)

In [ ]:

rouge = ROUGEScore()

mean_rouge_dict = {
    #"rouge1_fmeasure_mean" : np.mean([(rouge(shuffeld_reference[i],candidates)["rouge1_fmeasure"]) for i in range(len(shuffeld_reference))]),
    #"rouge1_precision_mean" :np.mean([rouge(shuffeld_reference[i],candidates)["rouge1_precision"] for i in range(len(shuffeld_reference))]),
    "rouge1_recall_mean" :np.mean([rouge(shuffeld_reference[i],candidates)["rouge1_recall"] for i in range(len(shuffeld_reference))]),
    #"rouge2_fmeasure_mean" :np.mean([rouge(shuffeld_reference[i],candidates)["rouge2_fmeasure"] for i in range(len(shuffeld_reference))]),
    #"rouge2_precision_mean" :np.mean([rouge(shuffeld_reference[i],candidates)["rouge2_precision"] for i in range(len(shuffeld_reference))]),
    "rouge2_recall_mean" :np.mean([rouge(shuffeld_reference[i],candidates)["rouge2_recall"] for i in range(len(shuffeld_reference))]),
    #"rougeL_fmeasure_mean" :np.mean([rouge(shuffeld_reference[i],candidates)["rougeL_fmeasure"] for i in range(len(shuffeld_reference))]),
    #"rougeL_precision_mean" :np.mean([rouge(shuffeld_reference[i],candidates)["rougeL_precision"] for i in range(len(shuffeld_reference))]),
    "rougeL_recall_mean" :np.mean([rouge(shuffeld_reference[i],candidates)["rougeL_recall"] for i in range(len(shuffeld_reference))]),
    #"rougeLsum_fmeasure_mean" :np.mean([rouge(shuffeld_reference[i],candidates)["rougeLsum_fmeasure"] for i in range(len(shuffeld_reference))]),
    #"rougeLsum_precision_mean" :np.mean([rouge(shuffeld_reference[i],candidates)["rougeLsum_precision"] for i in range(len(shuffeld_reference))]),
    "rougeLsum_recall_mean" : np.mean([rouge(shuffeld_reference[i],candidates)["rougeLsum_recall"] for i in range(len(shuffeld_reference))])
}

pprint(mean_rouge_dict)

{'rouge1_fmeasure_mean': 0.27499095,
 'rouge1_precision_mean': 0.2585979,
 'rouge1_recall_mean': 0.32948413,
 'rouge2_fmeasure_mean': 0.058802314,
 'rouge2_precision_mean': 0.05330027,
 'rouge2_recall_mean': 0.08263889,
 'rougeL_fmeasure_mean': 0.25817648,
 'rougeL_precision_mean': 0.24367064,
 'rougeL_recall_mean': 0.30791005,
 'rougeLsum_fmeasure_mean': 0.25817648,
 'rougeLsum_precision_mean': 0.24367064,
 'rougeLsum_recall_mean': 0.30791005}


## BLEURT Score

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-128")
model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-128")


In [ ]:
model.eval()

mean_score = []

for i in range(len(reference)):
  with torch.no_grad():
    scores = model(**tokenizer(reference[i], candidates[i], return_tensors='pt'))[0].squeeze()
    mean_score.append(scores.item())

print("mean BLEURT-Score:" ,np.mean(mean_score))


mean BLEURT-Score: -1.3870187749465306
